# Aviatrix Analytics Assignment

## Introduction and Load Data

Aviatrix is an online casino game. \
The player chooses how much to bet on landing a plane before it explodes. \
The plane is a customizable NFT. \
The plane collects experience and levels after every round.

This analysis will be composed of the follwoing stages:
- identification and description of data
- brief report with metrics that I consider most useful for description
- assessment of user engagment
- request amd recommendation for additional fields/events

In [68]:
import pandas as pd
import seaborn as sns
from pandas.tseries.offsets import Week

import numpy as np
import matplotlib.pyplot as plt

Below, we can see the data loaded from a csv file on the bets made in Aviatrix. \
Lets make an assumption that the column Currency represents the corresponding timezone or in other words country. \
Lets also assume that this is a table of bets and their information.

In [69]:
df = pd.read_csv('test_data.csv', parse_dates=['CreatedAt'])
df

,CreatedAt,Currency,UserId,PlatformId,MatchId,stake_currency,stake_eur,won_currency,won_eur
0,2023-03-20 15:01:13.304718058,KZT,wqsjm3ggga,platform_2,1013616,203.29,0.41,0.00,0.00
1,2023-03-13 13:40:07.304718058,TRY,enrlnd1u6y,platform_4,982714,1.88,0.09,5.79,0.29
2,2023-03-22 00:39:12.304718058,KZT,5nz03e471b,platform_1,1019784,605.22,1.21,0.00,0.00
3,2023-03-22 06:14:56.304718058,KZT,fiatnpu27d,platform_2,1020804,33.99,0.07,0.00,0.00
4,2023-03-01 14:32:41.304718058,KZT,1zx0tpf58u,platform_1,930347,18.93,0.04,0.00,0.00
...,...,...,...,...,...,...,...,...,...
99984,2023-03-27 05:50:36.304718058,KZT,yy04sqqhsn,platform_2,1042809,73.09,0.15,73.09,0.15
99985,2023-04-05 14:17:12.304718058,TRY,w86x6fnbx5,platform_2,1083864,5.73,0.27,0.00,0.00
99986,2023-03-11 19:34:34.304718058,KZT,w87kmjmmuc,platform_3,975059,26.22,0.05,0.00,0.00
99987,2023-03-31 21:33:30.304718058,KZT,bkwmgy6e6b,platform_1,1063176,88.01,0.18,0.00,0.00


## Identify and describe errors in the data

Types of data seem to be consistent with what they represent.

In [70]:
df.dtypes

CreatedAt         datetime64[ns]
Currency                  object
UserId                    object
PlatformId                object
MatchId                    int64
stake_currency           float64
stake_eur                float64
won_currency             float64
won_eur                  float64
dtype: object

From the very start we can see that the platform column registers platforms as listed below. \
This is not fitting for a good analysis, because one does not actually know from which platform a bet was made.

In [71]:
df['PlatformId'].unique()

array(['platform_2', 'platform_4', 'platform_1', 'platform_3'],
      dtype=object)

Lets try to see if there are any missing values in data. \
There is no missing data in the dataframe.

In [72]:
df[df.isna().any(axis=1)]

,CreatedAt,Currency,UserId,PlatformId,MatchId,stake_currency,stake_eur,won_currency,won_eur


There are also no duplicate entries.

In [73]:
df[df.duplicated()]

,CreatedAt,Currency,UserId,PlatformId,MatchId,stake_currency,stake_eur,won_currency,won_eur


The currencies seem to fit to the world definitions:
- KZT - Kazakhstan tenge
- TRY - TUrkey lire
- AZN - Azerbaijani manat
- INR - Indian rupee

In [74]:
df.Currency.unique()

array(['KZT', 'TRY', 'AZN', 'INR'], dtype=object)

There are no negative or zero values in the colummn MatchId.

In [75]:
df.query("MatchId <= 0")

,CreatedAt,Currency,UserId,PlatformId,MatchId,stake_currency,stake_eur,won_currency,won_eur


There are 2 rows that show stake currency being zero, which is an error because a bet cannot be zero.

In [76]:
df.query("stake_currency <= 0")

,CreatedAt,Currency,UserId,PlatformId,MatchId,stake_currency,stake_eur,won_currency,won_eur
5237,2023-03-10 19:18:40.304718058,AZN,0gdqrqabxe,platform_3,970695,0.0,0.0,0.00,0.0
87516,2023-04-01 07:08:16.304718058,AZN,pra6xdwjfs,platform_3,1064955,0.0,0.0,0.01,0.0


There are 262 rows that show stake euro to be zero. \
This is possible because a bet in a different currency can be lower than 1 cent euro.

In [77]:
df.query("stake_eur <= 0")

,CreatedAt,Currency,UserId,PlatformId,MatchId,stake_currency,stake_eur,won_currency,won_eur
777,2023-03-26 08:56:03.304718058,KZT,n9ker2uij0,platform_1,1038952,2.22,0.0,4.05,0.01
1301,2023-03-28 10:49:49.304718058,KZT,hqljaln82w,platform_1,1048103,2.20,0.0,0.00,0.00
1781,2023-02-26 12:20:18.304718058,KZT,5z9j6bp3ke,platform_3,917112,1.02,0.0,1.18,0.00
2089,2023-03-28 07:35:46.304718058,TRY,mww55mwkas,platform_3,1047526,0.08,0.0,0.08,0.00
2684,2023-03-20 05:49:15.304718058,KZT,t4au76z0un,platform_1,1011899,2.05,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...
98714,2023-03-05 11:46:57.304718058,KZT,86u9enqz6i,platform_1,947417,1.87,0.0,4.04,0.01
98791,2023-03-13 13:15:58.304718058,KZT,u3q5tv1xkf,platform_2,982637,1.70,0.0,1.98,0.00
99272,2023-03-14 21:28:59.304718058,TRY,vbycjxn3jv,platform_3,988571,0.05,0.0,0.14,0.01
99328,2023-03-27 02:20:10.304718058,KZT,lodhtrre41,platform_1,1042141,2.08,0.0,4.50,0.01


There are no negative values in columns euros won and currency won.

In [78]:
df.query("won_currency < 0 | won_eur < 0")

,CreatedAt,Currency,UserId,PlatformId,MatchId,stake_currency,stake_eur,won_currency,won_eur


There are no stakes that were larger than winnings.

In [79]:
df.query("stake_eur > won_eur & won_eur != 0")

,CreatedAt,Currency,UserId,PlatformId,MatchId,stake_currency,stake_eur,won_currency,won_eur


Errors found:
- platform names are incorrect
- there are 2 cases of zero amount bet

Lets exclude rows that have zero amount bet.

In [80]:
df = df.query("stake_currency != 0")

## A Brief Report: Description Metrics

The following metrics would describe existing bets data in the most useful way:
- daily active users (DAU) - how many unique users made bets each day
- stickiness in percentage (stickinesss_%) - DAU / WAU, how well users are retained over time
- average revenue per user (ARPU) - how much revenue user generates on average
- revenue - how much revenue is generated

In [113]:
df['dates'] = pd.to_datetime(df.CreatedAt).dt.date

min_date = pd.to_datetime(df.CreatedAt).min()
df['custom_weeks'] = np.floor((pd.to_datetime(df.CreatedAt) - min_date).dt.days / 7).astype(int)
df['custom_weeks'] = pd.to_datetime(min_date + pd.to_timedelta(df['custom_weeks'] * 7, unit='D')).dt.date

weekly_metrics = df.groupby(["custom_weeks"]) \
                    .agg(WAU=("UserId", "nunique")) \
                    .reset_index()

daily_metrics = df.groupby(["custom_weeks", "dates", "UserId"]) \
                    .agg(euros_bet=("stake_eur", "sum")) \
                    .reset_index() \
                        .groupby(["custom_weeks", "dates"]) \
                        .agg(
                            DAU=("UserId", "nunique"),
                            ARPU=("euros_bet", "mean"),
                            revenue=("euros_bet", "sum")
                        ) \
                        .reset_index()

metrics_by_weeks = daily_metrics.merge(weekly_metrics, on='custom_weeks')

metrics_by_weeks['DAU'] = metrics_by_weeks.DAU.round().astype("int")
metrics_by_weeks['ARPU'] = metrics_by_weeks.ARPU.round(2)

metrics_by_weeks['stickiness_%'] = ((metrics_by_weeks.DAU / metrics_by_weeks.WAU) * 100).round(1)

move_WAU = metrics_by_weeks.pop('WAU')
metrics_by_weeks.insert(3, move_WAU.name, move_WAU)

move_stickiness = metrics_by_weeks.pop('stickiness_%')
metrics_by_weeks.insert(4, move_stickiness.name, move_stickiness)

metrics_by_weeks['custom_weeks'] = pd.to_datetime(metrics_by_weeks.custom_weeks)
metrics_by_weeks = metrics_by_weeks.set_index(['custom_weeks', 'dates']).diff().reset_index()

metrics_by_weeks = metrics_by_weeks.drop('WAU', axis=1)

metrics_by_weeks.head()

C:\Users\User\AppData\Local\Temp\ipykernel_6488\890575593.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dates'] = pd.to_datetime(df.CreatedAt).dt.date
C:\Users\User\AppData\Local\Temp\ipykernel_6488\890575593.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['custom_weeks'] = np.floor((pd.to_datetime(df.CreatedAt) - min_date).dt.days / 7).astype(int)
C:\Users\User\AppData\Local\Temp\ipykernel_6488\890575593.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,custom_weeks,dates,DAU,stickiness_%,ARPU,revenue
0,2023-02-25,2023-02-25,NaN,NaN,NaN,NaN
1,2023-02-25,2023-02-26,-175.0,-2.8,0.87,625.75
2,2023-02-25,2023-02-27,234.0,3.7,-0.35,235.86
3,2023-02-25,2023-02-28,158.0,2.5,-0.24,61.34
4,2023-02-25,2023-03-01,-145.0,-2.3,0.13,-179.30


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

sns.lineplot(data=metrics_by_weeks, x='dates', y='DAU', ax=axes[0, 0])
axes[0, 0].set_title("Change in daily active users")

sns.histplot(data=df, x='dates', y='stickiness_%', ax=axes[0, 1])
axes[0, 1].set_title("Change in stickiness %")

sns.histplot(data=df, x='dates', hue='ARPU', ax=axes[1, 0])
axes[1, 0].set_title("Chane in Average Revenue per Users")

sns.histplot(data=df, x='won_eur', hue='PlatformId', alpha=0.3, bins=300, element="step", ax=axes[1, 1])
axes[1, 1].set_title("Euros won by platform")
axes[1, 1].set_ylim(0, 4000)
axes[1, 1].set_xlim(0, 40)

plt.show()

## User Engagement Assessment

Lets create a model where columns are metrics and rows are categories:
- num_users - number of unique users; indicates the size of the player base and tracking the changes over time would signal growth or decline of the player base
- num_bets - number of bets made; quantifies user activity and gives an overall level of user engagement
- total_euros_bet - total amount of euros bet; shows overall financial activity in the game and following changes would indicate trends in user spending behaviour
- avg_euros_bet - average amount of bets per user; helps understand the typical size of individual bets and can highlight variations in betting patterns

As we can see right away, Kazakhstan has the largest user base, user activity in terms of bets, and income stream. \
The country with the most second highest metrices in same categories is Turkey. \
Platform 1 is used only in Kazakhstan and it generates the most users, bets made, euros bet, and euros won more than any other platform

In [ ]:
by_category = df.groupby(["Currency", "PlatformId"]) \
                .agg(num_users=('UserId', 'nunique'),
                    num_bets=('MatchId', 'count'),
                    total_euros_bet=('stake_eur', 'sum'),
                    avg_euros_bet=('stake_eur', 'mean'))

by_category

num_users  num_bets  total_euros_bet  avg_euros_bet
Currency PlatformId                                                     
AZN      platform_2        579      3168          8666.27       2.735565
         platform_3       1153      7573         16008.28       2.113862
INR      platform_2       1468      7769         15086.72       1.941913
         platform_3       1642      7078          9721.52       1.373484
         platform_4         21        58            40.91       0.705345
KZT      platform_1      11150     40363         33813.58       0.837737
         platform_2       3148     14381         28607.60       1.989264
         platform_3        571      2288          1764.23       0.771080
TRY      platform_2        737      3720         11564.29       3.108680
         platform_3        524      1896          7706.08       4.064388
         platform_4       2443     11693         26863.32       2.297385

## Additional Data

In order to assess user engagement more thoroughly, the following metrics would be required:
- session duration - events that begin and end user's session
- achievement unlocks - user_id and timestamp when each achievement was unlocked after level and experience progression; also the name and description of each unique achievement
- NFT customization - events that indicate each time a player engages in changing his/her plana